In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
%pwd

'/content'

In [2]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks

/content/gdrive/MyDrive/Colab Notebooks


In [3]:
%ls

 audio/
 audio-0.8.2.tar.gz
'Copy of mozilla_deepspeech_api_notebook.ipynb'
 deepspeech-0.8.2-models/
 geckodriver.exe
 geckodriver.log
 household_power_consumption.txt
 MyFirstColabNotebook.ipynb
 product_detection_model.ipynb
 products_images/
 scraping_product_images.ipynb
 Stage_5_Project.ipynb


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import os
import matplotlib.pyplot as plt
import glob

%matplotlib inline

In [5]:
batch_size = 1

image_gen_train = ImageDataGenerator(rescale=1./255, 
                                     rotation_range=15, 
                                     width_shift_range=.15, 
                                     height_shift_range=.15, 
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     zoom_range=0.2
                                    )


train_data_gen = image_gen_train.flow_from_directory(directory='/content/gdrive/MyDrive/Colab Notebooks/products_images/train',
                                                     class_mode='categorical', 
                                                     target_size=(256,256),
                                                     batch_size=batch_size)

Found 310 images belonging to 5 classes.


In [6]:
train_data_gen.class_indices

{'iphone12_pro_max': 0,
 'jbl_charge3': 1,
 'nintendo_switch': 2,
 'ps4_controller': 3,
 'yeezy_boost_350': 4}

In [7]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(directory='/content/gdrive/MyDrive/Colab Notebooks/products_images/validate', 
                                                   class_mode='categorical', 
                                                   target_size=(256,256),
                                                   batch_size=batch_size)

Found 113 images belonging to 5 classes.


In [8]:
val_data_gen.class_indices

{'iphone12_pro_max': 0,
 'jbl_charge3': 1,
 'nintendo_switch': 2,
 'ps4_controller': 3,
 'yeezy_boost_350': 4}

In [9]:
transfer_learning_layer = tf.keras.applications.InceptionResNetV2(include_top=False, 
                                                      weights='imagenet', 
                                                      input_shape=(256,256,3)
                                                     )

In [10]:
transfer_learning_layer.trainable = False

In [11]:
model = Sequential([
    transfer_learning_layer,
    Dropout(0.2),
    Flatten(),
    Dense(5, activation='softmax')
])

In [12]:
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy']
             )

In [13]:
epochs = 4
history = model.fit(train_data_gen, 
                    epochs=epochs, 
                    validation_data=val_data_gen)

Epoch 1/4
310/310 [==============================] - 30s 62ms/step - loss: 8.2827 - accuracy: 0.7065 - val_loss: 4.2020 - val_accuracy: 0.8230
Epoch 2/4
310/310 [==============================] - 16s 51ms/step - loss: 5.5486 - accuracy: 0.8161 - val_loss: 2.2944 - val_accuracy: 0.9381
Epoch 3/4
310/310 [==============================] - 15s 49ms/step - loss: 4.2711 - accuracy: 0.8613 - val_loss: 1.9550 - val_accuracy: 0.9292
Epoch 4/4
310/310 [==============================] - 15s 50ms/step - loss: 5.8033 - accuracy: 0.8774 - val_loss: 1.7358 - val_accuracy: 0.9469


In [14]:
model.save('/content/gdrive/MyDrive/Colab Notebooks/extracted_product_detection_model')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/extracted_product_detection_model/assets
